<a href="https://colab.research.google.com/github/adamott/AdamTraining/blob/main/Day9_chatbot_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install transformers gradio --quiet

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr

model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
chatbot_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def format_chat_history(history):
    formatted = []
    for user_msg, bot_msg in history:
        formatted.append(f"User: {user_msg}{tokenizer.eos_token}")
        formatted.append(f"Bot: {bot_msg}{tokenizer.eos_token}")
    return "\n".join(formatted)

def chatbot(user_input, history):
    history = history or []

    # Format conversation history with EOS tokens
    conversation = format_chat_history(history)
    prompt = f"{conversation}User: {user_input}{tokenizer.eos_token}\nBot:"

    # Generate response
    response = chatbot_pipeline(
        prompt,
        max_new_tokens=200,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.85,
        repetition_penalty=1.15,
        no_repeat_ngram_size=2,
        truncation=True
    )[0]['generated_text']

    # Extract bot reply
    try:
        # Get the portion after the last Bot: occurrence
        bot_reply = response.split(prompt)[1]  # Get newly generated part
        bot_reply = bot_reply.split(f"{tokenizer.eos_token}")[0]  # Split at EOS
        bot_reply = bot_reply.split("Bot:")[-1].strip()
    except Exception as e:
        bot_reply = "Hmm, let's talk about something else!"

    # Update conversation history (keep last 5 exchanges)
    history.append((user_input, bot_reply))
    history = history[-5:]

    return "", history

with gr.Blocks() as demo:
    gr.Markdown("# Chat with DialoGPT")
    chatbot_interface = gr.Chatbot(height=500)
    user_input = gr.Textbox(label="Type your message", placeholder="Say something...")
    clear_button = gr.Button("Clear Chat")

    user_input.submit(
        chatbot,
        inputs=[user_input, chatbot_interface],
        outputs=[user_input, chatbot_interface]
    )
    clear_button.click(lambda: [], None, chatbot_interface, queue=False)

demo.launch()

Device set to use cpu
<ipython-input-32-13465c84477a>:53: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_interface = gr.Chatbot(height=500)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d911ea26a133dfdfae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
